In [0]:
# %pip install -q -r requirements.txt

In [0]:
import torch
from torch.utils.data import DataLoader
from src.data_loader import SingleImageDataset, TRAIN_TRANSFORM
from src.unet_model import build_unet, train_unet
import random, numpy as np

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = SingleImageDataset(
    "data/stockton_1.png",
    mask_path="data/stockton_1_mask.png",
    transform=TRAIN_TRANSFORM
)
dataloader = DataLoader(dataset, batch_size=1)

model = build_unet(classes=3, in_channels=3)
model, epoch_losses = train_unet(
    model, dataloader, device,
    epochs=30,
    save_path="checkpoints/unet_last.pt"
)

In [0]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(epoch_losses)+1), epoch_losses, marker='o')
plt.title("Training Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("CrossEntropy Loss")
plt.grid()
plt.show()

In [0]:
import numpy as np
np.__version__